<a href="https://colab.research.google.com/github/chiranjeet14/ML_Journey/blob/master/Hackerearth-Predict-the-genetic-disorders/4_genetic_testing_autoML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!python3 -m pip install --upgrade pip > /dev/null
!pip3 install scikit-learn --upgrade > /dev/null
!pip3 install pandas --upgrade > /dev/null
!pip3 install numpy --upgrade > /dev/null

In [ ]:
!curl https://raw.githubusercontent.com/automl/auto-sklearn/master/requirements.txt | xargs -n 1 -L 1 pip3 install > /dev/null
!pip3 install auto-sklearn > /dev/null
!pip3 install auto-sklearn --upgrade > /dev/null

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   249  100   249    0     0    849      0 --:--:-- --:--:-- --:--:--   846


In [ ]:
import pandas as pd
import numpy as np
import io
import gc
import time
from pprint import pprint
from datetime import date

# settings
import warnings
warnings.filterwarnings("ignore")
gc.enable()

In [ ]:
# connect to google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
gDrivePath = '/content/drive/MyDrive/Datasets/Hackerearth_genetic_testing/dataset/'

In [ ]:
df_train = pd.read_csv(gDrivePath+'train_preprocessed.csv')
df_test = pd.read_csv(gDrivePath+'test_preprocessed.csv')

In [ ]:
df_train.sample(3)

,Patient Age,Genes in mother's side,Inherited from father,Maternal gene,Paternal gene,Blood cell count (mcL),Mother's age,Father's age,Status,Respiratory Rate (breaths/min),Heart Rate (rates/min,Follow-up,Gender,Birth asphyxia,Autopsy shows birth defect (if applicable),Folic acid details (peri-conceptional),H/O serious maternal illness,H/O radiation exposure (x-ray),H/O substance abuse,Assisted conception IVF/ART,History of anomalies in previous pregnancies,No. of previous abortion,Birth defects,White Blood cell count (thousand per microliter),Blood test result,Symptom 1,Symptom 2,Symptom 3,Symptom 4,Symptom 5,Genetic Disorder,Disorder Subclass
461,13.0,1.0,0.0,0.584759,0.0,4.805470,40.0,41.91325,1.0,2.00000,2.000000,1.510835,2.0,1.0,2.0,2.0,2.0,2.280013,2.0,1.0,2.0,3.0,1.00000,5.836821,3.0,1.0,0.546332,1.0,0.0,0.0,Mitochondrial genetic inheritance disorders,Mitochondrial myopathy
9681,10.0,0.0,1.0,1.000000,0.0,4.722113,20.0,54.00000,1.0,1.52843,1.492788,1.000000,3.0,1.0,4.0,1.0,1.0,4.000000,3.0,2.0,2.0,1.0,1.49806,4.354147,2.0,1.0,1.000000,1.0,0.0,1.0,Single-gene inheritance diseases,Cystic fibrosis
2717,14.0,0.0,0.0,0.446868,0.0,4.896845,36.0,30.00000,0.0,2.00000,2.000000,2.000000,3.0,2.0,3.0,2.0,1.0,2.256200,4.0,2.0,1.0,1.0,1.00000,3.452216,4.0,1.0,0.000000,0.0,0.0,0.0,Single-gene inheritance diseases,Hemochromatosis


In [ ]:
df_train['GeneticDisorder-DisorderSubclass'] = df_train['Genetic Disorder'] + '<->' + df_train['Disorder Subclass']

In [ ]:
target_labels = df_train['GeneticDisorder-DisorderSubclass'].values

df_train.drop(['Genetic Disorder','Disorder Subclass', 'GeneticDisorder-DisorderSubclass'], axis=1, inplace=True)
df_test.drop(['Genetic Disorder','Disorder Subclass', 'GeneticDisorder-DisorderSubclass'], axis=1, inplace=True, errors='ignore')

In [ ]:
# classification split for genetic_disorder_labels
from sklearn.model_selection import train_test_split
X_train, X_cv, y_train, y_cv = train_test_split(df_train, target_labels, test_size=0.1, random_state=50)

In [ ]:
from autosklearn.classification import AutoSklearnClassifier
from autosklearn.metrics import f1_weighted

cls = AutoSklearnClassifier(memory_limit=None, metric=f1_weighted)
cls.fit(X_train, y_train)

AutoSklearnClassifier(memory_limit=None, metric=f1_weighted,
                      per_run_time_limit=360)

In [ ]:
run_key = list(cls.automl_.runhistory_.data.keys())[2]
run_value = cls.automl_.runhistory_.data[run_key]

print(f'Configuration ID: {run_key.config_id}')
print(f'Instance: {run_key.instance_id}')
print(f'Seed: {run_key.seed}')
print(f'Budget: {run_key.budget}')

print(cls.automl_.runhistory_.ids_config[run_key.config_id])

Configuration ID: 3
Instance: {"task_id": "ef855d42-f22c-11eb-85a8-0242ac1c0002"}
Seed: 0
Budget: 0.0
Configuration:
  balancing:strategy, Value: 'none'
  classifier:__choice__, Value: 'lda'
  classifier:lda:shrinkage, Value: 'auto'
  classifier:lda:tol, Value: 0.018821286956948503
  data_preprocessing:categorical_transformer:categorical_encoding:__choice__, Value: 'no_encoding'
  data_preprocessing:categorical_transformer:category_coalescence:__choice__, Value: 'minority_coalescer'
  data_preprocessing:categorical_transformer:category_coalescence:minority_coalescer:minimum_fraction, Value: 0.017839122072146087
  data_preprocessing:numerical_transformer:imputation:strategy, Value: 'most_frequent'
  data_preprocessing:numerical_transformer:rescaling:__choice__, Value: 'minmax'
  feature_preprocessor:__choice__, Value: 'kitchen_sinks'
  feature_preprocessor:kitchen_sinks:gamma, Value: 0.9170797845861378
  feature_preprocessor:kitchen_sinks:n_components, Value: 1867



In [ ]:
# cls.get_models_with_weights()

In [ ]:
predictions_test = cls.predict(df_test)

In [ ]:
predictions_test

array(['Multifactorial genetic inheritance disorders<->Diabetes',
       'Single-gene inheritance diseases<->Tay-Sachs',
       'Mitochondrial genetic inheritance disorders<->Mitochondrial myopathy',
       ...,
       'Mitochondrial genetic inheritance disorders<->Mitochondrial myopathy',
       'Multifactorial genetic inheritance disorders<->Diabetes',
       'Multifactorial genetic inheritance disorders<->Diabetes'],
      dtype=object)

In [ ]:
predictions_genetic_disorder_test = []
predictions_disorder_subclass_test = []

for myString in predictions_test:
  genetic_disorder, disorder_subclass = myString.split('<->')
  predictions_genetic_disorder_test.append(genetic_disorder) 
  predictions_disorder_subclass_test.append(disorder_subclass)

print("length genetic_disorder list:", len(predictions_genetic_disorder_test))
print("length disorder_subclass list:", len(predictions_disorder_subclass_test))

length genetic_disorder list: 9465
length disorder_subclass list: 9465


In [ ]:
read = pd.read_csv(gDrivePath + 'test.csv')
read.shape

(9465, 43)

In [ ]:
submission = pd.DataFrame({
        "Patient Id": read["Patient Id"],
        "Genetic Disorder": predictions_genetic_disorder_test,
        "Disorder Subclass": predictions_disorder_subclass_test,
    })

In [ ]:
submission.head()

,Patient Id,Genetic Disorder,Disorder Subclass
0,PID0x4175,Multifactorial genetic inheritance disorders,Diabetes
1,PID0x21f5,Single-gene inheritance diseases,Tay-Sachs
2,PID0x49b8,Mitochondrial genetic inheritance disorders,Mitochondrial myopathy
3,PID0x2d97,Mitochondrial genetic inheritance disorders,Leigh syndrome
4,PID0x58da,Single-gene inheritance diseases,Cystic fibrosis


In [ ]:
submission.to_csv('submission.csv', index=False)